# Importing modules and settings

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
from matplotlib.pyplot import rc_context

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

General settings of Scanpy

In [ ]:
sc.settings.verbosity = 4
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')


In [ ]:
umap_cmap = sns.light_palette('xkcd:medium blue', as_cmap = True)

# Declaring the input and output files

In [ ]:
name_of_analysis = 'pristina_atlas'

In [ ]:
sc.settings.figdir = './figures/'+name_of_analysis+'_doublets_included'

In [ ]:
results_file_with_doublets = './'+name_of_analysis+'_doublets_included.h5ad'

In [ ]:
results_file_prefiltering = './'+name_of_analysis+'_prefiltering.h5ad'

In [ ]:
adata = sc.read_10x_mtx(
    "./pristina/",
    var_names='gene_symbols',
    cache=True)

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata.var_names

In [ ]:
adata.var

In [ ]:
adata

# Annotating the samples

In [ ]:
adata.obs

In [ ]:
sample_ids = adata.obs.index.str.slice(start = 0, stop = -24)

In [ ]:
sample_ids

In [ ]:
sample_ids_split = sample_ids.str.split("_")

In [ ]:
sample_ids_split

In [ ]:
set(sample_ids)

In [ ]:
list(set(sample_ids))

In [ ]:
experiments = list(set(item[0]+'_'+item[1] for item in sample_ids_split))

In [ ]:
experiments

In [ ]:
libraries = list(set(item[0]+'_'+item[1]+'_'+item[2] for item in sample_ids_split))

In [ ]:
libraries

In [ ]:
for word in experiments:
        cellfilt = adata.obs.index.str.contains(word+'_')
        adata.obs.loc[cellfilt, "Experiment"] = word

In [ ]:
for word in libraries:
        cellfilt = adata.obs.index.str.contains(word+'_')
        adata.obs.loc[cellfilt, "Library"] = word

In [ ]:
adata.obs

In [ ]:
adata.obs.iloc[32500]

# **Scrublet**

In [ ]:
sc.external.pp.scrublet(adata, batch_key = 'Experiment', threshold = 0.14)

In [ ]:
sc.external.pl.scrublet_score_distribution(adata)

In [ ]:
adata.obs

In [ ]:
adata.obs['predicted_doublet'].value_counts()

In [ ]:
adata

In [ ]:
pd.pivot_table(adata.obs, index = 'predicted_doublet')

## **Reading SOLO output**

In [ ]:
solo = pd.read_excel('./solo_ratio_multimappers 10.xlsx', index_col = 'Unnamed: 0')

In [ ]:
solo.index = solo.index.str.slice(stop=-2)

In [ ]:
solo

In [ ]:
solo['doublet']

In [ ]:
adata.obs['solo_doublet'] = solo['doublet']

In [ ]:
adata.obs['solo_singlet'] = solo['singlet']

In [ ]:
adata.obs

In [ ]:
pd.pivot_table(adata.obs, index = 'predicted_doublet')

In [ ]:
th_solo = 1.50

In [ ]:
adata.obs[adata.obs['solo_doublet'] > th_solo]

In [ ]:
adata.obs['solo_top'] = "singlet"

In [ ]:
adata.obs

In [ ]:
filt_solo_top = adata.obs['solo_doublet'] > th_solo

In [ ]:
adata.obs[filt_solo_top]

In [ ]:
adata.obs.loc[filt_solo_top, 'solo_top'] = 'doublet'

In [ ]:
adata.obs

In [ ]:
adata.obs['solo_top'].value_counts()

In [ ]:
doub_cols = adata.obs[['predicted_doublet', 'solo_top']]

In [ ]:
pd.pivot_table(doub_cols, index = 'solo_top')

In [ ]:
filsolo = doub_cols['solo_top'] == 'doublet'

In [ ]:
filscrub = doub_cols['predicted_doublet'] == True

In [ ]:
doub_cols[filsolo & filscrub]

In [ ]:
adata.obs[filsolo & filscrub]

# Exporting h5ad without doublets

In [ ]:
adata.obs[~(filsolo | filscrub)]

In [ ]:
adata.obs

In [ ]:
adata.obs.loc[(adata.obs['predicted_doublet'] == False) & (adata.obs['solo_top'] == 'singlet'), 'doublets'] = 'singlet'

In [ ]:
adata.obs.loc[(adata.obs['predicted_doublet'] == True) & (adata.obs['solo_top'] == 'singlet'), 'doublets'] = 'doublet scrublet'

In [ ]:
adata.obs.loc[(adata.obs['predicted_doublet'] == False) & (adata.obs['solo_top'] == 'doublet'), 'doublets'] = 'doublet solo'

In [ ]:
adata.obs.loc[(adata.obs['predicted_doublet'] == True) & (adata.obs['solo_top'] == 'doublet'), 'doublets'] = 'doublet both'

In [ ]:
adata.obs['doublets'] = adata.obs['doublets'].astype('category')

In [ ]:
adata.obs['doublets'].cat.reorder_categories(['singlet', 'doublet solo', 'doublet scrublet', 'doublet both'], inplace=True)

In [ ]:
adata.obs['doublets'].value_counts()

In [ ]:
adata[~(filsolo | filscrub)]

In [ ]:
adata_no_doublets = adata[~(filsolo | filscrub)]

In [ ]:
adata_no_doublets.write(results_file_prefiltering)

# Preprocessing

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20 )

In [ ]:
sc.pp.filter_cells(adata, min_counts=50)
sc.pp.filter_cells(adata, min_genes= 50)
sc.pp.filter_genes(adata, max_counts = 1000000)

In [ ]:
adata.var

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, log = True)

In [ ]:
adata

# Matrix slicing

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 700, :]

In [ ]:
adata = adata[adata.obs.total_counts < 900, :]

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

# Normalization and log transformation

The following 2 functions normalise and log transform the matrix

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

# Selecting highly variable genes

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes = 18000)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata

In [ ]:
adata.raw.var

# Scaling the data

In [ ]:
sc.pp.scale(adata, zero_center=False)

# Performing the PCA and kNN analysis

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps = 150)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=150, log=True)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=45, n_pcs=105)

In [ ]:
sc.tl.umap(adata, min_dist=0.5, spread = 1, alpha = 1, gamma = 1.0)

In [ ]:
sc.pl.umap(adata)

In [ ]:
tr = 'PrileiEVm023936t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm008309t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm011741t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm021316t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm022250t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm000325t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm013699t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
tr = 'PrileiEVm020595t1'
sc.pl.umap(adata, color=tr, title = tr, color_map = umap_cmap, save = tr+'.pdf')

In [ ]:
sc.pl.umap(adata, color='PrileiEVm023936t1', title = 'PrileiEVm023936t1', color_map = umap_cmap)

In [ ]:
sc.pl.umap(adata, color='PrileiEVm008309t1', title = 'PrileiEVm008309t1', color_map = umap_cmap)

In [ ]:
sc.pl.umap(adata, color='PrileiEVm021040t1', title = 'PrileiEVm021040t1', color_map = umap_cmap)

In [ ]:
sc.pl.umap(adata, color='PrileiEVm017310t1', title = 'PrileiEVm017310t1', color_map = umap_cmap)

In [ ]:
sc.pl.umap(adata, color='PrileiEVm000199t1', title = 'PrileiEVm000199t1', color_map = umap_cmap)

In [ ]:
sc.pl.umap(adata, color='PrileiEVm019805t1', title = 'PrileiEVm019805t1', color_map = umap_cmap)

In [ ]:
with rc_context({'figure.figsize': (12, 12)}):
    sc.pl.umap(adata, color='doublets', size = 20, groups = ['doublet solo', 'doublet scrublet', 'doublet both'], palette = ['lightgrey', 'pink', 'orange', 'red'],
              save = 'doublet.pdf')
    

# Clustering

Indicate how many different resolutions you want to try

In [ ]:
resolutions = [1, 2, 3, 4]

In [ ]:
for i in resolutions:
    sc.tl.leiden(adata, resolution = i, key_added = 'leiden_with_doublet_'+str(i))
    sc.pl.umap(adata, color='leiden_with_doublet_'+str(i))

In [ ]:
leiden_names = adata.obs.columns[adata.obs.columns.str.contains('leiden')].to_list()

In [ ]:
leiden_names

In [ ]:
clusteringlayer = 'leiden_with_doublet_3'

In [ ]:
sc.pl.umap(adata, color=leiden_names, legend_loc = 'on data', legend_fontsize = 10)

In [ ]:
for leiden_i in leiden_names:
    with rc_context({'figure.figsize': (25, 25)}):
        sc.pl.umap(adata, color=leiden_i, legend_loc='on data', title=str(leiden_i), size = 50, frameon=False,
                  save = leiden_i+'.pdf')

In [ ]:
percs_leiden_1 = {}
for cl in adata.obs['leiden_with_doublet_1'].cat.categories:
    cts = adata.obs[adata.obs['leiden_with_doublet_1'] == cl]['doublets'].value_counts()
    percs = cts / cts.sum() * 100
    percs_leiden_1[cl] = percs
percs_df_leiden_1 = pd.DataFrame(percs_leiden_1)

In [ ]:
with rc_context({'figure.figsize': (25, 5)}):
    percs_df_leiden_1.transpose().plot(kind='bar', stacked=True, color=['lightgrey', 'pink', 'orange', 'red'])
    plt.legend(loc='lower right')
    plt.savefig('./figures/pristina_atlas_doublets_included/percs_leiden_1.pdf')

In [ ]:
percs_leiden_2 = {}
for cl in adata.obs['leiden_with_doublet_2'].cat.categories:
    cts = adata.obs[adata.obs['leiden_with_doublet_2'] == cl]['doublets'].value_counts()
    percs = cts / cts.sum() * 100
    percs_leiden_2[cl] = percs
percs_df_leiden_2 = pd.DataFrame(percs_leiden_2)

In [ ]:
with rc_context({'figure.figsize': (25, 5)}):
    percs_df_leiden_2.transpose().plot(kind='bar', stacked=True, color=['lightgrey', 'pink', 'orange', 'red'])
    plt.legend(loc='lower right')
    plt.savefig('./figures/pristina_atlas_doublets_included/percs_leiden_2.pdf')

In [ ]:
percs_leiden_3 = {}
for cl in adata.obs['leiden_with_doublet_3'].cat.categories:
    cts = adata.obs[adata.obs['leiden_with_doublet_3'] == cl]['doublets'].value_counts()
    percs = cts / cts.sum() * 100
    percs_leiden_3[cl] = percs
percs_df_leiden_3 = pd.DataFrame(percs_leiden_3)

In [ ]:
with rc_context({'figure.figsize': (25, 5)}):
    percs_df_leiden_3.transpose().plot(kind='bar', stacked=True, color=['lightgrey', 'pink', 'orange', 'red'])
    plt.legend(loc='lower right')
    plt.savefig('./figures/pristina_atlas_doublets_included/percs_leiden_3.pdf')

In [ ]:
percs_leiden_4 = {}
for cl in adata.obs['leiden_with_doublet_4'].cat.categories:
    cts = adata.obs[adata.obs['leiden_with_doublet_4'] == cl]['doublets'].value_counts()
    percs = cts / cts.sum() * 100
    percs_leiden_4[cl] = percs
percs_df_leiden_4 = pd.DataFrame(percs_leiden_4)

In [ ]:
with rc_context({'figure.figsize': (25, 5)}):
    percs_df_leiden_4.transpose().plot(kind='bar', stacked=True, color=['lightgrey', 'pink', 'orange', 'red'])
    plt.legend(loc='lower right')
    plt.savefig('./figures/pristina_atlas_doublets_included/percs_leiden_4.pdf')

In [ ]:
adata.write(results_file_with_doublets)